In [ ]:
#!/usr/bin/env python3

from statistics import mean
from math import sqrt

Define a BayesianAgent.

In [ ]:
class BayesianAgent:
    """
    bayesian agent
    """

    def __init__(self, data, update_size=10):
        self.size = len(data)           # size of dataset: reliability of his private signal (list)
        self.mean = mean(data)        # private signal: random assigned (list)
        self.sqr_mean = mean([x**2 for x in data])

The bayesian market maker has a prior belief about the data that he or she observes. 
In our implementation, we separated the data into two parts. The first part (the first 20 points) is the part which agents use to form their priors.
Our agents will observe data that are of random size from the first 20 points, and form an initial mean (an intial private signal) for the two securities.

In [ ]:
        self.decision = [False, False]    # sell(False) or buy(True) security (list of bool)
        self.security_amount = [0, 0]     # two security amounts (list)
        self.update_size = update_size
        # self.budget = None: might be used in future

In [ ]:
    def update_param(self, data):
            """
            update private signal based on observed samples
            :param data: the data observed by the agent after entering the market
                         length of the data should be self.update_size
            :return:
            """
            self.mean = (self.mean * self.size + self.update_size * mean(data))/(self.update_size + self.size)
            sqr_data_mean = mean([x**2 for x in data])
            self.sqr_mean = (self.sqr_mean * self.size + self.update_size * sqr_data_mean)/(self.update_size + self.size)
            self.size += self.update_size

Once the agents have observed new data, they can update their belief using the equation (15) in Information Aggregation in Exponential Family Markets.
The size of the data an agent sees each time is fixed to 10.

In [ ]:
    @staticmethod
    def calculate_outstanding_shares(current_market_price):
        """
        Calculate the number of outstanding shares using the current market price
        :return: theta_1, theta_2
        """
        p_1, p_2 = current_market_price[0], current_market_price[1]
        theta_1 = p_1/(p_2-p_1**2)
        theta_2 = 0.5/(p_1**2-p_2)
        return theta_1, theta_2

Here the agent can secretly calculate the number of shares of each security in the market according to the current price of each security.

In [ ]:
    def calculate_belief(self, num_trades, current_market_price, security_type="X"):
        """
         Calculate belief of agent based on market price
        :param security_type: takes two values, "X" or "X_sqr"
        :param num_trades: n
        :param current_market_price: niu
        :return: belief
        """
        if security_type == "X":
            return (num_trades * current_market_price + self.mean) / (num_trades + 1)
        elif security_type == "X_sqr":
            return (num_trades * current_market_price + self.sqr_mean) / (num_trades + 1)

The agent forms a private belief based on the data points that he or she sees. However, when the agent trades, he weighs in the belief formed by other agents.
Here the number of trades is published by the market maker. The current market price represent the common belief of the agents in the market.
The resultant value is the value that an agent uses to decide the amount of shares he or she would trade.

In [ ]:
def calculate_shares_to_buy(self, num_trades, current_market_price):
        """
        Calculate the number of shares agents should purchase to move the current market price to his or her expectation
        :param  num_trades: the number of trades the market maker has seen in market
        :param  current_market_price: list of prices
        :param  security_type: 0 is X security, 1 is X**2 security
        :return:
        """
        theta_1, theta_2 = self.calculate_outstanding_shares(current_market_price)
        belief_1 = self.calculate_belief(num_trades, current_market_price[0], "X")
        belief_2 = self.calculate_belief(num_trades, current_market_price[1], "X_sqr")
        delta_1 = belief_1 / (belief_2 - belief_1**2) - theta_1
        delta_2 = - theta_2 - 1/(2 * (belief_2 - belief_1**2))
        return delta_1, delta_2

The way an agent calculates the amount of shares he or she wants to buy is using the price formula. Here we have derived the formula for the agent to move the market price to fit his or her calculated belief (after weighing in the market price)

In [ ]:
def update_security(self, delta_1, delta_2):
        """
        Update the number of securities an agent has based on whether he or she buys or sells securities
        :return:
        """
        # A positive delta stands for a buy, a negative delta stands for a sell
        self.security_amount[0] += delta_1
        self.security_amount[1] += delta_2